In [7]:
# Import necessary libraries
import math
import sys
import os
from jupylet.sprite import Sprite
from jupylet.label import Label
from jupylet.app import App

# Setup system path
sys.path.insert(0, os.path.abspath('./..'))

# Initialize the app and sprites
app = App(width=800, height=800)
background = Sprite('images.png', scale=8)
ship = Sprite('player.png', x=app.width/2, y=app.height/2, scale=0.5)
alien = Sprite('alien.jpg', x=100, y=100, scale=0.1)

# Initialize key states and movement speed
W = A = S = D = False
Stealth = Run = False
player_speeds = {'run': 45, 'stealth': 15, 'walk': 25}
alien_speed = 60

# Initialize labels to display coordinates
player_position_label = Label(f'Player X: {ship.x:.2f}, Y: {ship.y:.2f}', font_size=24, color='black', x=10, y=app.height - 30)
alien_target_label = Label(f'Alien Target X: {alien.x:.2f}, Y: {alien.y:.2f}', font_size=24, color='black', x=10, y=app.height - 60)

# Update ship position based on user input
@app.run_me_every(1/60)
def update_ship(ct, dt):
    global speed

    # Adjust speed based on player state
    speed = player_speeds['stealth'] if Stealth else player_speeds['run'] if Run else player_speeds['walk']
    ship.image = 'player_s.png' if Stealth else 'player_r.png' if Run else 'player.png'

    # Move ship based on key states
    ship.x += (-speed * dt) if A else (speed * dt) if D else 0
    ship.y += (speed * dt) if W else (-speed * dt) if S else 0

    # Ensure ship stays within window bounds
    ship.wrap_position(app.width, app.height)

# Update alien target position based on distance to the ship
@app.run_me_every(1/60)
def update_alien_target(ct, dt):
    dx, dy = ship.x - alien.x, ship.y - alien.y
    distance = math.sqrt(dx**2 + dy**2)
    update_interval = max(0.1, 5.0 * math.exp(-0.01 * distance))

    # Update target position of alien
    if not hasattr(alien, 'last_update_time') or (ct - alien.last_update_time > update_interval):
        alien.target_x, alien.target_y = ship.x, ship.y
        alien.last_update_time = ct

# Update alien position to move towards the target coordinates
@app.run_me_every(1/60)
def update_alien(ct, dt):
    if hasattr(alien, 'target_x') and hasattr(alien, 'target_y'):
        dx, dy = alien.target_x - alien.x, alien.target_y - alien.y
        distance = math.sqrt(dx**2 + dy**2)
        if distance > 0:
            alien.x += (dx / distance) * alien_speed * dt
            alien.y += (dy / distance) * alien_speed * dt

# Update labels every frame to show current player and alien target positions
@app.run_me_every(1/60)
def update_labels(ct, dt):
    player_position_label.text = f'Player X: {ship.x:.2f}, Y: {ship.y:.2f}'
    alien_target_label.text = f'Alien Target X: {alien.target_x:.2f}, Y: {alien.target_y:.2f}' if hasattr(alien, 'target_x') else f'Alien Target X: N/A, Y: N/A'

# Handle keyboard events to control ship
@app.event
def key_event(key, action, modifiers):
    global W, A, D, S, Stealth, Run
    keys = app.window.keys
    pressed = action == keys.ACTION_PRESS

    if key == keys.W: W = pressed
    elif key == keys.A: A = pressed
    elif key == keys.D: D = pressed
    elif key == keys.S: S = pressed
    elif key == keys.SPACE: Run = pressed
    elif key == keys.C and pressed: Stealth = not Stealth

# Render the game
@app.event
def render(ct, dt):
    app.window.clear()
    background.draw()
    ship.draw()
    alien.draw()
    player_position_label.draw()
    alien_target_label.draw()

# Run the application
app.run()


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…